# 第十一章 特征工程之数据预处理（11.5）

# 11.5 特征筛选：WOE值与IV值

11.5.3 WOE值与IV值的代码实现

1.数据分箱

In [1]:
# 首先通过如下代码构造数据：
import pandas as pd
data = pd.DataFrame([[22,1],[25,1],[20,0],[35,0],[32,1],[38,0],[50,0],[46,1]], columns=['年龄', '是否违约'])
data

,年龄,是否违约
0,22,1
1,25,1
2,20,0
3,35,0
4,32,1
5,38,0
6,50,0
7,46,1


In [2]:
# 有了数据之后，根据“年龄”这一特征变量进行数据分箱，代码如下：
data_cut = pd.cut(data['年龄'], 3)
data_cut

0    (19.97, 30.0]
1    (19.97, 30.0]
2    (19.97, 30.0]
3     (30.0, 40.0]
4     (30.0, 40.0]
5     (30.0, 40.0]
6     (40.0, 50.0]
7     (40.0, 50.0]
Name: 年龄, dtype: category
Categories (3, interval[float64]): [(19.97, 30.0] < (30.0, 40.0] < (40.0, 50.0]]

2.统计各个分箱样本总数、坏样本数和好样本数

In [3]:
# 统计总客户数
cut_group_all = data['是否违约'].groupby(data_cut).count()
# 统计违约客户
cut_y = data['是否违约'].groupby(data_cut).sum()
# 统计未违约客户
cut_n = cut_group_all - cut_y

In [4]:
# 这里展示下cut_group_all的结果，如下所示：
cut_group_all

年龄
(19.97, 30.0]    3
(30.0, 40.0]     3
(40.0, 50.0]     2
Name: 是否违约, dtype: int64

In [5]:
# 通过2.2.1节相关知识点将cut_group_all、cut_y、cut_n进行汇总，代码如下，这里我们将违约客户命名为“坏样本”，非违约客户命名为“好样本”。
df = pd.DataFrame()  # 创建一个空DataFrame用来汇总数据
df['总数'] = cut_group_all
df['坏样本'] = cut_y
df['好样本'] = cut_n
df

,总数,坏样本,好样本
年龄,,,
"(19.97, 30.0]",3,2,1
"(30.0, 40.0]",3,1,2
"(40.0, 50.0]",2,1,1


3.统计各分箱中坏样本比率和好样本比率

In [6]:
# 计算坏样本%和好样本%
df['坏样本%'] = df['坏样本'] / df['坏样本'].sum()
df['好样本%'] = df['好样本'] / df['好样本'].sum()
df

,总数,坏样本,好样本,坏样本%,好样本%
年龄,,,,,
"(19.97, 30.0]",3,2,1,0.50,0.25
"(30.0, 40.0]",3,1,2,0.25,0.50
"(40.0, 50.0]",2,1,1,0.25,0.25


4.计算WOE值

In [7]:
import numpy as np
df['WOE'] = np.log(df['坏样本%'] / df['好样本%'])
df

,总数,坏样本,好样本,坏样本%,好样本%,WOE
年龄,,,,,,
"(19.97, 30.0]",3,2,1,0.50,0.25,0.693147
"(30.0, 40.0]",3,1,2,0.25,0.50,-0.693147
"(40.0, 50.0]",2,1,1,0.25,0.25,0.000000


此外，我们在11.5.1节第一部分也讲过，在实际应用中，我们不希望WOE值出现无穷大（这样会导致之后计算的IV值也变为无穷大，丧失了IV值的意义），但是有的时候可能由于数据特殊性及分箱的原因，它还是出现了WOE值为无穷大的情况（某个分箱中只含有一种类别的数据），此时解决办法是当WOE值为无穷大时，将它替换为0，代码如下：

In [8]:
df = df.replace({'WOE': {np.inf: 0, -np.inf: 0}})

5.计算IV值

In [9]:
df['IV'] = df['WOE'] * (df['坏样本%'] - df['好样本%'])
df

,总数,坏样本,好样本,坏样本%,好样本%,WOE,IV
年龄,,,,,,,
"(19.97, 30.0]",3,2,1,0.50,0.25,0.693147,0.173287
"(30.0, 40.0]",3,1,2,0.25,0.50,-0.693147,0.173287
"(40.0, 50.0]",2,1,1,0.25,0.25,0.000000,0.000000


In [10]:
iv = df['IV'].sum()
print(iv)

0.34657359027997264


整理上面计算WOE值和IV值的内容，完整代码如下所示：

In [11]:
# 1.构造数据
import pandas as pd
data = pd.DataFrame([[22,1],[25,1],[20,0],[35,0],[32,1],[38,0],[50,0],[46,1]], columns=['年龄', '是否违约'])

# 2.数据分箱
data_cut = pd.cut(data['年龄'], 3)

# 3.统计各个分箱样本总数、坏样本数和好样本数并汇总数据
# 统计总客户数
cut_group_all = data['是否违约'].groupby(data_cut).count()
# 统计违约客户
cut_y = data['是否违约'].groupby(data_cut).sum()
# 统计未违约客户
cut_n = cut_group_all - cut_y
# 汇总基础数据
df = pd.DataFrame()  # 创建一个空DataFrame用来汇总数据
df['总数'] = cut_group_all
df['坏样本'] = cut_y
df['好样本'] = cut_n

# 4.统计坏样本%和好样本%
df['坏样本%'] = df['坏样本'] / df['坏样本'].sum()
df['好样本%'] = df['好样本'] / df['好样本'].sum()

# 5.计算WOE值
import numpy as np
df['WOE'] = np.log(df['坏样本%'] / df['好样本%'])
df = df.replace({'WOE': {np.inf: 0, -np.inf: 0}})  # 替换可能存在的无穷大

# 6.计算各个分箱的IV值
df['IV'] = df['WOE'] * (df['坏样本%'] - df['好样本%'])

# 7.汇总各个分箱的IV值，获得特征变量的IV值
iv = df['IV'].sum()
print(iv)

0.34657359027997264


11.5.4 案例实战：客户流失预警模型的IV值计算

In [12]:
# 将上面的内容首先定义为一个函数
import pandas as pd
import numpy as np

def cal_iv(data, cut_num, feature, target):
    # 1.数据分箱
    data_cut = pd.cut(data[feature], cut_num)

    # 2.统计各个分箱样本总数、坏样本数和好样本数
    cut_group_all = data[target].groupby(data_cut).count()  # 总客户数
    cut_y = data[target].groupby(data_cut).sum()  # 坏样本数
    cut_n = cut_group_all - cut_y  # 好样本数
    # 汇总基础数据
    df = pd.DataFrame()  # 创建一个空DataFrame用来汇总数据
    df['总数'] = cut_group_all
    df['坏样本'] = cut_y
    df['好样本'] = cut_n

    # 3.统计坏样本%和好样本%
    df['坏样本%'] = df['坏样本'] / df['坏样本'].sum()
    df['好样本%'] = df['好样本'] / df['好样本'].sum()

    # 4.计算WOE值
    df['WOE'] = np.log(df['坏样本%'] / df['好样本%'])
    df = df.replace({'WOE': {np.inf: 0, -np.inf: 0}}) 

    # 5.计算各个分箱的IV值
    df['IV'] = df['WOE'] * (df['坏样本%'] - df['好样本%'])

    # 6.汇总各个分箱的IV值，获得特征变量的IV值
    iv = df['IV'].sum()
    
    print(iv)

In [13]:
# 有了上面的自动计算IV值的函数后，通过如下代码来读取客户流失预警模型中的相关数据：
data = pd.read_excel('股票客户流失.xlsx')
data.head()

,账户资金（元）,最后一次交易距今时间（天）,上月交易佣金（元）,本券商使用时长（年）,是否流失
0,22686.5,297,149.25,0,0
1,190055.0,42,284.75,2,0
2,29733.5,233,269.25,0,1
3,185667.5,44,211.50,3,0
4,33648.5,213,353.50,0,1


In [14]:
# 我们利用刚刚编好的函数进行第一个特征变量“账户资金（元）”的IV值计算，代码如下：
cal_iv(data, 4, '账户资金（元）', '是否流失')

0.15205722409339645


In [15]:
for i in data.columns[:-1]:
    print(i + '的IV值为：')
    cal_iv(data, 4, i, '是否流失')  # 调用函数

账户资金（元）的IV值为：
0.15205722409339645
最后一次交易距今时间（天）的IV值为：
0.2508468300174099
上月交易佣金（元）的IV值为：
0.30811632146662304
本券商使用时长（年）的IV值为：
0.6144219248359752
